# 65-2-alternate-shift-schedules-by-cell
# using the new objects

In [43]:
from collections import OrderedDict as OD
from gpkit import Variable, Model, units
from gpx.offshift import OffShiftCell
from gpx.manufacturing import QNACell, FabLine
from gpx.primitives import Process

In [44]:
processes = [
    Process(time=1*units('hr'), cv=1),
    Process(time=2*units('hr'), cv=1),
    Process(time=1*units('hr'), cv=1),
]

In [45]:
cellnames = ['first', 'second', 'third']

cells = OD()

for i in range(len(cellnames)):
    cells[cellnames[i]] = QNACell(processes[i], return_process=True)

In [46]:
ton = Variable('T_{on}', 12, 'hrs', 'on shift time')
toff = Variable('T_{off}', 8, 'hrs', 'off shift time')

In [47]:
oscell = OffShiftCell(cells['second'], cells['third'], ton, toff)

In [48]:
cells['second'] = oscell

In [49]:
oscell.substitutions

{OffShiftCell3.\chi_{cv}: 1,
 OffShiftCell3.\eta_t: 1,
 OffShiftCell3.\frac{T_{on} + T_{off}}{T_{on}}: <function gpx.offshift.OffShiftCell.setup.<locals>.<lambda>(m)>,
 OffShiftCell3.\text{Prob}[T_{off}]: <function gpx.offshift.OffShiftCell.setup.<locals>.<lambda>(m)>,
 T_{off}: 8}

In [50]:
line = FabLine(list(cells.values()), return_cells=True)

In [51]:
constr = []
prodqty = Variable('N_{production}', 'count', 'daily production quantity')
constr.append(prodqty <= line.lam*ton)

# maximize daily quantity
cost = 1/prodqty*100 + cells['second'].m

In [52]:
constr.extend(oscell.get_line_constraints(line))

In [53]:
constr.append(line)

In [54]:
m = Model(cost, constr)

In [55]:
m.substitutions.update({
    cells['first'].m : 1,
    cells['second'].m : 1,
    cells['third'].m : 1,
    toff : 8*units('hr'),
    ton : 12*units('hr'),
    line.L : 20,
})

In [56]:
print(m.solve().table())

Using solver 'mosek_cli'
 for 33 free variables
  in 45 posynomial inequalities.
Solving took 0.093 seconds.

Optimal Cost
------------
 12.11

Free Variables
--------------
N_{production} : 8.998   [count]    daily production quantity

               | FabLine3
             W : 26.67   [hr]       Total flow time
       \lambda : 0.7498  [count/hr] Line Continuous Production Rate

               | Process10
        \sigma : 60      [min]      standard deviation of process time

               | Process11
        \sigma : 120     [min]      standard deviation of process time

               | Process12
        \sigma : 60      [min]      standard deviation of process time

               | QNACell10
             W : 1066    [min]      Total flow time through cell
            Wq : 945.5   [min]      Expected queueing time
        \alpha : 0.1002             (1-rho)
       \lambda : 0.7498  [count/hr] Production rate
           c2a : 0.9773             Arrival coefficient of variation squ